In [2]:
from preprocess import *
from df_edit import *

In [ ]:
!pip install googletrans==4.0.0-rc1

In [1]:
from googletrans import Translator

translator = Translator()

In [13]:
print(translator.translate('호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 명칭을 통일하였다.', src='ko', dest='en'))

Translated(src=ko, dest=en, text=Honam-based, the right futures, alternative and Democratic Peace, were worthy of the end of the twist, and unified the name (tentative)., pronunciation=None, extra_data="{'confiden...")


In [3]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-3-3c150f134808>:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


#### https://me2.kr/0j4cp 보고 따라하고 Google이 나오면 됩니다.

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome import service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

options = Options()
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems

s=Service('/opt/ml/chromedriver')
driver = webdriver.Chrome(options=options, service=s)
driver.get('http://google.com/')
myPageTitle = driver.title
print(myPageTitle)
driver.quit()

Google


In [4]:
train_df = pd.read_csv('/opt/ml/dataset/train/train.csv')
train_df_2 = better_df(train_df)
train_df_2 = Preprocess(train_df_2)

In [5]:
from load_data import *
train_dataset = load_data("/opt/ml/dataset/train/train.csv")


In [6]:
len(train_dataset)

32423

In [136]:
# index 잘못 되어 있는 지 확인
only_BMP_pattern = re.compile("["
    u"\U00010000-\U0010FFFF"  #BMP characters 이외
                       "]+", flags=re.UNICODE)
for id in train_df_2['id']:
    # while loop 돌면서 문장안에 BMP 문자 이외의 단어를 발견 할 때마다 space * 단어길이로 치환함.
    # 한 문장 안의 여러 특수 문자 있을 때에도 성공적으로 제거하며, obj,sbj index 건드리지 않음.
    while only_BMP_pattern.search(train_df_2['sentence'].loc[id]) != None:
        start_index = only_BMP_pattern.search(train_df_2['sentence'].loc[id]).start()
        end_index = only_BMP_pattern.search(train_df_2['sentence'].loc[id]).end()
        space_len = end_index-start_index
        train_df_2['sentence'].loc[id] = train_df_2['sentence'].loc[id][:start_index] + ' ' * space_len + train_df_2['sentence'].loc[id][end_index:]

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [137]:
# index 잘못 되어 있는 지 확인
for i in range(len(train_df_2)):
    if train_df_2['sentence'].iloc[i][train_df_2['subj_start'].iloc[i]:train_df_2['subj_end'].iloc[i]+1] != train_df_2['subj_word'].iloc[i]:
        print(train_df_2['sentence'].iloc[i])
        print(train_df_2['subj_word'].iloc[i])

In [117]:
train_df_2['sentence'].loc[3926]

'혜령군 이정 또는 이지(惠寧君 李  또는 李祉, 1407년 ~ 1440년)은 조선의 왕족으로 태종의 넷째 서자이다.'

In [79]:
train_df_2['sentence'].loc[31939]

'마가다(   )는 기원전 684년부터 기원전 321년까지 북인도의 비하르 남부를 중심으로 존재하던 십육대국 국가이다.'

In [138]:
# augmentation data 용 id : index dictionary제작
sen_dict = train_df_2.to_dict()
sub_ent_dict = {}
obj_ent_dict = {}

for idx in sen_dict['sentence'].keys():
    sub_ent_dict[idx]=(sen_dict['subj_start'][idx],sen_dict['subj_end'][idx])
    obj_ent_dict[idx]=(sen_dict['obj_start'][idx],sen_dict['obj_end'][idx])

In [60]:
# encode dict에 encode된 문장 저장
sub_code = "VKV" # entity가 사라지는 것을 방지
obj_code = "VJV"
encode_dict={}
for idx in sen_dict['sentence'].keys():
    sub_start = sub_ent_dict[idx][0]
    sub_end = sub_ent_dict[idx][1]
    obj_start = obj_ent_dict[idx][0]
    obj_end = obj_ent_dict[idx][1]
    if sub_start<obj_start:
        encode_dict[idx]=(sen_dict['sentence'][idx][:sub_start]+sub_code+sen_dict['sentence'][idx][sub_end+1:obj_start]+obj_code+sen_dict['sentence'][idx][obj_end+1:])
    else :
        encode_dict[idx]=(sen_dict['sentence'][idx][:obj_start]+obj_code+sen_dict['sentence'][idx][obj_end+1:sub_start]+sub_code+sen_dict['sentence'][idx][sub_end+1:])

In [61]:
# 인코딩 확인
len(encode_dict)

32423

In [62]:
# 200자가 넘길 때마다 처음 만나는 space에서 ... 삽입
for key,item in encode_dict.items():
    if len(item)>200:
        quotioent = len(item) // 200
        new_line_idx_list=[]
        for i in range(quotioent):
            new_line_idx = 200*(i+1)
            while new_line_idx < len(item) and item[new_line_idx] != ' ':
                new_line_idx += 1
            new_line_idx_list.append(new_line_idx)
        for n_idx in new_line_idx_list:
            item = item[:n_idx]+'...'+item[n_idx+1:]
        encode_dict[key]=item

# 왜 이 짓을 하는가? 파파고는 한 문장당 200자만 번역이 가능하다 ..
        

In [63]:
len(encode_dict.keys())

32423

In [18]:
# 분업용 index 나누는 함수
split_size = len(encode_dict.keys())//6
split_key = []
for i in range(6):
    if i != 5:
        split_key.append(list(encode_dict.keys())[i*split_size:(i+1)*split_size])
    else:
        split_key.append(list(encode_dict.keys())[i*split_size:])

In [19]:
total_size=0
for i in range(6):
    print(len(split_key[i]))
    total_size += len(split_key[i])
print(total_size)

5403
5403
5403
5403
5403
5408
32423


In [20]:
chk = 0
for key in split_key[0]: # 277,3548 번 제거된 것 확인
    if key != chk:
        print(key)
        chk +=1
    chk +=1

278
3548


In [304]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import numpy as np
from tqdm.auto import tqdm
from datetime import datetime
import pickle
import itertools
# 크롬창(웹드라이버) 열기
options = Options()
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems
s=Service('/opt/ml/chromedriver')
driver = webdriver.Chrome(options=options, service=s)

    



In [163]:
import re
# 이전에 만든거 삭제 X
data = '안녕하세요 반갑습니다🐶'
for i in range(6):
        for key in split_key[i]:
                data = encode_dict[key]
                only_BMP_pattern = re.compile("["
                        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                                        "]+", flags=re.UNICODE)
                if only_BMP_pattern.findall(data) != []:
                        print(key)            
                        print(only_BMP_pattern.findall(data))# BMP characters만

3926
['𥘺']
31939
['𑀫𑀕𑀥']


In [64]:
import re

# data = '안녕하세요 반갑습니다🐶'
# 제대로 삭제되었는지 확인
for i in range(6):
        for key in split_key[i]:
                data = encode_dict[key]
                only_BMP_pattern = re.compile("["
                        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                                        "]+", flags=re.UNICODE)
                if only_BMP_pattern.findall(data) != []:
                        print(key)            
                        print(only_BMP_pattern.findall(data))# BMP characters만

In [307]:
# 파파고 페이지 접속

target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))


i = 0 # 여기에 자신이 배정받은 번호 입력
start_index = split_key[i][0]
last_index = split_key[i][-1]
kor_to_eng_dict={}

for key in split_key[i]:
    data = encode_dict[key]
    # 입력창에 검색어 입력
    backtrans=''
    # element=WebDriverWait(driver, 10).until(target_present)
    driver.get('https://papago.naver.com/?sk=ko&tk=en')
    time.sleep(0.1)
    driver.find_element(By.XPATH,value='//*[@id="sourceEditArea"]/label').send_keys(data)
    driver.find_element(By.XPATH,value='//*[@id="btnTranslate"]').click()
    time.sleep(1.5)
    backtrans = WebDriverWait(driver, 20).until(target_present).text
    # backtrans = driver.find_element(By.XPATH,value='//*[@id="txtTarget"]').text
    kor_to_eng_dict[key]=backtrans

with open(f'kor_eng_{start_index}_to_{last_index}.pickle', 'wb') as f:
    pickle.dump(kor_to_eng_dict, f, pickle.HIGHEST_PROTOCOL)

In [345]:
from selenium.webdriver.common.keys import Keys

target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
# eng_to_kor_dict={}
final_index = list(kor_to_eng_dict.keys())[-1]

for idx,(key,data) in enumerate(kor_to_eng_dict.items()):
    # 입력창에 검색어 입력
    if idx >=77:
        if data != '':
            backtrans=''
            # element=WebDriverWait(driver, 10).until(target_present)
            driver.get('https://papago.naver.com/?sk=en&tk=ko')
            time.sleep(0.1)
            driver.find_element(By.XPATH,value='//*[@id="sourceEditArea"]/label').send_keys(data)
            driver.find_element(By.XPATH,value='//*[@id="btnTranslate"]').send_keys(Keys.ENTER)
            time.sleep(1.5)
            backtrans = WebDriverWait(driver, 10).until(target_present).text
            # backtrans = driver.find_element(By.XPATH,value='//*[@id="txtTarget"]').text
            eng_to_kor_dict[key]=backtrans
            if (idx+1)%100 == 1 :
                start_index = key
            if (idx+1)%100 == 0 or key == final_index :
                last_index = key
                with open(f'eng_kor_{start_index}_to_{last_index}.pickle', 'wb') as f:
                    pickle.dump(eng_to_kor_dict, f, pickle.HIGHEST_PROTOCOL)
                eng_to_kor_dict ={}

driver.close()

KeyboardInterrupt: 

In [1]:
eng_to_kor_dict

NameError: name 'eng_to_kor_dict' is not defined

In [21]:
eng_to_kor_dict

NameError: name 'eng_to_kor_dict' is not defined

In [119]:
import pickle
aug_dict ={}
start_idx_list = [0,501,1001,1501,2001,2501,3001,3501,4002,4502,5002,5405,5805,6205,6605,7006,7407,7807,8207,8608,9009,9409,9809,10211,10611,10815,11831,12007,12815,13315,13815,14316,14817,15317,15818,16221,18127,19222,19986,21644,21644,21644,21644,21644,21644,21644,21644,21644,21644,27042,27042,27042,27042,27042,27042,27042,27042,27042,27042,27042]
end_idx_list = [500,1000,1500,2000,2500,3000,3500,4001,4501,5001,5404,5804,6204,6604,7005,7406,7806,8206,8607,9008,9408,9808,10210,10610,10814,11830,12006,12814,13314,13814,14315,14816,15316,15817,16220,18126,19221,19985,21627,27054,27055,27056,27057,27058,27059,27060,27061,27062,27063,32469,32470,32471,32472,32473,32474,32475,32476,32477,32478,32479]
for start,end in zip(start_idx_list,end_idx_list):
    with open(f'./translate_result/eng_kor_{start}_to_{end}.pickle','rb') as f:
        data = pickle.load(f)
        aug_dict.update(data)
len(aug_dict)

31252

In [120]:
import re
score = 0
aug_subj_idx_dict = {}
aug_obj_idx_dict = {}
adj_aug_dict = {}

# 번역 시 붙어있는 경우 고려
sub_pattern = re.compile("[Vv][Kk][Vv]")
obj_pattern = re.compile("[Vv][Jj][Vv]")
s_pattern_1 = re.compile("[Vv][Kk][Vv][Jj][Vv]")
s_pattern_2 = re.compile("[Vv][Jj][Vv][Kk][Vv]")
s_pattern_3 = re.compile("[Vv][Kk][Vv][Vv][Jj][Vv]")
s_pattern_4 = re.compile("[Vv][Jj][Vv][Vv][Kk][Vv]")

for idx,item in aug_dict.items():
    if item != '' and idx not in [21757,22090,22222,22258,22641,22772,24788,25094,25368,26044]: # 번역이 안된 경우 제외
        if sub_pattern.search(item) != None and obj_pattern.search(item) != None:
            # print(item)
            item=re.sub(s_pattern_1,'VKV VJV',item)
            item=re.sub(s_pattern_2,'VJV VKV',item)
            item=re.sub(s_pattern_3,'VKV VJV',item)
            item=re.sub(s_pattern_4,'VJV VKV',item)
            score += 1

            sub_start_idx = sub_pattern.search(item).start()
            obj_start_idx = obj_pattern.search(item).start()
            if sub_start_idx < obj_start_idx:
                
                sub_start_idx = sub_pattern.search(item).start()
                item=re.sub(sub_pattern,sen_dict['subj_word'][idx],item)
                
                aug_subj_idx_dict[idx]=((sub_start_idx,sub_start_idx+len(sen_dict['subj_word'][idx])-1))

                obj_start_idx = obj_pattern.search(item).start()
                item=re.sub(obj_pattern,sen_dict['obj_word'][idx],item)
                aug_obj_idx_dict[idx]=((obj_start_idx,obj_start_idx+len(sen_dict['obj_word'][idx])-1))

            else :
                obj_start_isdx = obj_pattern.search(item).start()
                item=re.sub(obj_pattern,sen_dict['obj_word'][idx],item)
                aug_obj_idx_dict[idx]=((obj_start_idx,obj_start_idx+len(sen_dict['obj_word'][idx])-1))

                sub_start_idx = sub_pattern.search(item).start()
                item=re.sub(sub_pattern,sen_dict['subj_word'][idx],item)
                aug_subj_idx_dict[idx]=((sub_start_idx,sub_start_idx+len(sen_dict['subj_word'][idx])-1))

                

            adj_aug_dict[idx]=item
            # print(item)
print("restoration percent : ", score/(31252))
# aug_subj_idx_dict

restoration percent :  0.7844617944451555


In [121]:
len(aug_subj_idx_dict)

24516

In [122]:
for idx,item in adj_aug_dict.items():
    if idx not in aug_subj_idx_dict.keys():
        print(adj_aug_dict[idx][aug_subj_idx_dict[idx][0]:aug_subj_idx_dict[idx][1]+1])

In [123]:
for idx,item in adj_aug_dict.items():
    if idx not in aug_obj_idx_dict.keys():
        print(adj_aug_dict[idx][aug_obj_idx_dict[idx][0]:aug_obj_idx_dict[idx][1]+1])

In [130]:

for idx,item in adj_aug_dict.items():
    if idx in aug_subj_idx_dict.keys():
        insert_data={'id':idx+32470,
                    'subj_type':sen_dict['subj_type'][idx],
                    'subj_word':sen_dict['subj_word'][idx],
                    'subj_start':aug_subj_idx_dict[idx][0],
                    'subj_end':aug_subj_idx_dict[idx][1],
                    'obj_type':sen_dict['obj_type'][idx],
                    'obj_word':sen_dict['obj_word'][idx],
                    'obj_start':aug_obj_idx_dict[idx][0],
                    'obj_end':aug_obj_idx_dict[idx][1],
                    'sentence':adj_aug_dict[idx],
                    'label':sen_dict['label'][idx],
                    'label_num':sen_dict['label_num'][idx]}
        train_df_2 = train_df_2.append(insert_data,ignore_index=True)

In [131]:
len(train_df_2) #추가됬는지 확인
# 약간 문제가 되는게 번역이 안되는 문장에 대해 index와 id가 안맞는 일이 발생함 조금 수정하면 될 것 같은데 문제 없는 이상 이대로 하겠음.

56939

In [133]:
train_df_2.tail(10)

,id,subj_type,subj_word,subj_start,subj_end,obj_type,obj_word,obj_start,obj_end,sentence,label,label_num
56929,64925,ORG,비나신,68,70,ORG,베트남,63,65,"김홍균 연구원은 ""1회 손익에는 신규 수주와 건조손실준비금 118억원, 결함수리준비금 125억원이 반영됐다""며 ""베트남현대비나신조선소 건조 증가로 전분기 기준 매출이 8.4% 증가했다""고 말했다.",org:place_of_headquarters,7
56930,64926,PER,신대철,6,8,DAT,1967년 2월 16일,10,21,"대한민국의 신대철(1967년 2월 16일 申大澈,)이 뮤지션.",per:date_of_birth,25
56931,64927,ORG,정읍시,4,6,NOH,1100대,59,63,21일 정읍시에 따르면 정부는 올해 노후 경유차 조기 폐차 지원 사업에 17억6880만원의 예산을 확보해 1100대에 보조금을 지급할 예정이다.,no_relation,0
56932,64928,ORG,스티어 프라이데이 나이트,61,73,POH,Stir Friday Night,43,59,아시아계 미국인들로 구성된 그는 대학 스케치 코미디 그룹을 졸업한 직후에 '(Stir Friday Night 스티어 프라이데이 나이트)'가 되었다.,org:alternate_names,5
56933,64929,ORG,삼성 라이온즈,44,50,DAT,2011년,52,56,와 메이저 리그 야구 경력을 10게임 2012년에서원 사서 비싸게.12월 2일 삼성 라이온즈 2011년.,no_relation,0
56934,64931,PER,밀러드 필모어,55,61,ORG,휘그당,14,16,1852년 미국 대선에서 휘그당는 미-멕시코 전쟁의 영웅 테일러 장군의 사망으로 대통령직을 승계한 밀러드 필모어 현 대통령의 지명을 거부했다.,per:employee_of,6
56935,64933,PER,루이 14세,26,31,PER,루이 13세,19,24,사촌오빠는 그의 삼촌이 내 남편인 루이 13세 루이 14세이다.,per:other_family,13
56936,64934,PER,이창진,13,15,POH,외야수,9,11,"기아, 타이거즈 외야수 이창진 롯데부문. 매장 광주는 앞서 시상식을 열었던 9월 월간 MVP LG 28일에 선정된 상을 수여할 것으로 예상된다.광주, 한국",per:title,4
56937,64936,PER,최시형,14,16,PER,손병희,31,33,"지도자로 Beopo, 다시최시형 seobyeonghak,손병희 bukjjeop고 다시, 전봉준,gimgaenam seojangok namjeop으로 나뉠 것이다.",per:colleagues,14
56938,64939,ORG,화순군,0,2,PER,구충곤,7,9,"화순군(군수 구충곤)는 동면장 20여명이 COVID-19 예방을 위해 버스 승강장과 공중화장실, 오동리 천운아파트 등 다중이용시설 소독에 자원했다고 17일 밝혔다.",org:top_members/employees,1


In [134]:
train_df_2.to_csv("aug_train_final.csv", index=False)

In [142]:
encode_dict=sen_dict['sentence']

In [143]:
for key,item in encode_dict.items():
    if key == 0:
        print(item)

〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.


In [148]:
!pip install sentencepiece

In [2]:
from backtranslation import BackTranslation_ko

2022-04-05 10:25:45.973178: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [12]:
from koeda import EasyDataAugmentation


eda = EasyDataAugmentation(
    morpheme_analyzer="Okt", alpha_sr=0.3, alpha_ri=0.3, alpha_rs=0.3, prob_rd=0.3
)

# text = "아버지가 방에 들어가신다"

# result = eda(text)
# print(result)
# # 아버지가 정실에 들어가신다

# result = eda(text, p=(0.9, 0.9, 0.9, 0.9), repetition=2)
# print(result)
# # ['아버지가 객실 아빠 안방 방에 정실 들어가신다', '아버지가 탈의실 방 휴게실 에 안방 탈의실 들어가신다']

TypeError: startJVM() got an unexpected keyword argument 'classpath'

In [14]:
from konlpy.tag import Okt
okt = Okt()
text = "아 나는 역시 밤에 일이 잘된다."

print(okt.morphs(text, stem=True))

TypeError: startJVM() got an unexpected keyword argument 'classpath'

In [15]:
from konlpy.tag import Okt
okt = Okt()

TypeError: startJVM() got an unexpected keyword argument 'classpath'